In [ ]:
import math
import gzip
import pickle
import argparse
import numpy as np

import torch 
from torch import nn
from torch.autograd import Variable 
from torch import Tensor as t
import torch.nn.functional as F
from torch.distributions import Normal
from torch.optim import Adam 
import torch.utils.data as data_utils

from pytorch_util import MLP
from reparameterize import Nreparameterize
from reparameterize import SO3reparameterize
from pytorch_util import logsumexp, load_mnist_data, n2p

from vae import VAE

import ipympl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline 
#%matplotlib notebook

### Generate Random Point and Rotation
- Either create a Gaussian point cloud, or generate a random line.
- Generate a random rotation by sampling from 'some' distribution and applying exponential map.

In [ ]:
def randomR():
    v = np.random.normal(0,1,(1,3))
    v_x = SO3reparameterize._expmap_rodrigues(n2p(v)).squeeze()
    return v_x 

def gen_point_pair(mu = 0., std = 1., n=2, line=False):
    if line:
        x = np.tile(np.random.normal(mu, std, (1, 3)), [n,1])
        x[:,0] = x[:,0] + np.linspace(-2,2,n)
    else:
        x = np.random.normal(mu,std,(n,3))
    x = n2p(x)
    R = randomR()
    x_rot = x @ R
    return x, x_rot, R

### Plot original and rotated point

In [ ]:
def plot_3d(v, c='red', m='o'):
    for i in range(v.size()[0]):
        xs, ys, zs = v.data.numpy()[i,:]
        ax.scatter(xs, ys, zs, c=c, marker=m)


x, x_rot, R_x = gen_point_pair(0., 1., 10, line=True)

print ((x - (x_rot @ R_x.transpose(0,1))).sum().data.numpy() < 1e-4)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
    
plot_3d(x)
plot_3d(x_rot, c='blue')
    
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
ax.set_xlim((-4,4))
ax.set_ylim((-4,4))
ax.set_zlim((-4,4));